In [ ]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

# ---------------------------------------------------
# Parameters
# ---------------------------------------------------
N = 256        # total length of signal (1 second duration → fs_cont = N Hz)
k = 4          # sparsity in Fourier domain
np.random.seed(0)

# Generate sparse Fourier coefficients
freqs = np.random.choice(N//2, k, replace=False)
coeffs = np.zeros(N, dtype=complex)
coeffs[freqs] = np.random.randn(k) + 1j*np.random.randn(k)
x_true = np.real(np.fft.ifft(coeffs)) * N
t = np.arange(N)

f_max = freqs.max()   # maximum frequency in original signal

# ---------------------------------------------------
# Sinc reconstruction
# ---------------------------------------------------
def sinc_reconstruct(t, idx_samples, x_samples, N):
    T = N/len(idx_samples)  # effective period
    return np.sum(x_samples * np.sinc((t[:,None] - idx_samples[None,:]) / T), axis=1)

# ---------------------------------------------------
# Compressed sensing recovery
# ---------------------------------------------------
def compressed_sensing_reconstruct(x, M):
    idx = np.sort(np.random.choice(N, M, replace=False))
    y = x[idx]

    # Measurement operator (rows of IFFT matrix)
    F = np.fft.ifft(np.eye(N)) * N
    A = F[idx,:]

    # Solve: min ||a||_1  s.t. A a = y
    a = cp.Variable(N, complex=True)
    objective = cp.Minimize(cp.norm1(a))
    constraints = [A @ a == y]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    a_hat = a.value
    x_hat = np.real(np.fft.ifft(a_hat)) * N
    return x_hat, idx

# ---------------------------------------------------
# Interactive update
# ---------------------------------------------------
def update(M=40):
    fs_eff = M  # effective sampling frequency

    # --- Uniform sampling ---
    step = N // M
    idx_uniform = np.arange(0, N, step)[:M]
    x_samples = x_true[idx_uniform]
    x_sinc = sinc_reconstruct(t, idx_uniform, x_samples, N)

    # --- Compressed sensing ---
    x_cs, idx_random = compressed_sensing_reconstruct(x_true, M)

    # --- Build Plotly figure ---
    fig = go.Figure()

    # Original
    fig.add_trace(go.Scatter(x=t, y=x_true, mode='lines',
                             name="Original signal", line=dict(color='black')))

    # Sinc reconstruction
    fig.add_trace(go.Scatter(x=t, y=x_sinc, mode='lines',
                             name="Sinc reconstruction (uniform)", 
                             line=dict(color='blue', dash='dash')))

    # CS reconstruction
    fig.add_trace(go.Scatter(x=t, y=x_cs, mode='lines',
                             name=f"CS reconstruction (random)", 
                             line=dict(color='orange', dash='dash')))

    # Uniform samples (blue dots)
    fig.add_trace(go.Scatter(x=idx_uniform, y=x_true[idx_uniform], mode='markers',
                             name="Uniform samples", marker=dict(color='blue', size=8)))

    # Random samples (red dots)
    fig.add_trace(go.Scatter(x=idx_random, y=x_true[idx_random], mode='markers',
                             name="Random samples", marker=dict(color='red', size=8)))

    # Top annotation (Nyquist info)
    fig.add_annotation(
        x=0.5, y=max(x_true)+0.5,
        text=f"M = {M} → fs = {fs_eff} Hz | Nyquist = {2*f_max} Hz | f_max = {f_max} Hz",
        showarrow=False, font=dict(family="Times New Roman", color="green", size=12),
        xref="paper", yref="y"
    )

    # Bottom annotation (number of sampling points)
    fig.add_annotation(
        x=0.5, y=-0.25, xref="paper", yref="paper",
        text=f"Number of sampling points (uniform & CS) = {M}",
        showarrow=False, font=dict(family="Times New Roman", color="darkblue", size=12)
    )

    # Apply Times New Roman globally
    fig.update_layout(
        title=f"Uniform vs. CS Reconstruction (M={M} samples)",
        title_font=dict(family="Times New Roman", size=20),
        xaxis_title="Time index",
        yaxis_title="Amplitude",
        font=dict(family="Times New Roman", size=16),
        height=650,
        margin=dict(b=80),  # leave space for bottom annotation
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

# ---------------------------------------------------
# Interactive widget
# ---------------------------------------------------
interact(update, M=IntSlider(min=10, max=N, step=5, value=40))


interactive(children=(IntSlider(value=40, description='M', max=256, min=10, step=5), Output()), _dom_classes=(…

<function __main__.update(M=40)>

In [11]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
from ipywidgets import interact, IntSlider

# ---------------------------------------------------
# Parameters
# ---------------------------------------------------
N = 256        # total length of signal (1 second duration → fs_cont = N Hz)
k = 4          # sparsity in Fourier domain
np.random.seed(0)

# Generate sparse Fourier coefficients
freqs = np.random.choice(N//2, k, replace=False)
coeffs = np.zeros(N, dtype=complex)
coeffs[freqs] = np.random.randn(k) + 1j*np.random.randn(k)
x_true = np.real(np.fft.ifft(coeffs)) * N
t = np.arange(N)

f_max = freqs.max()   # maximum frequency in original signal

# ---------------------------------------------------
# Sinc reconstruction
# ---------------------------------------------------
def sinc_reconstruct(t, idx_samples, x_samples, N):
    T = N/len(idx_samples)  # effective period
    t_samples_idx = idx_samples
    return np.sum(x_samples * np.sinc((t[:,None] - t_samples_idx[None,:]) / T), axis=1)

# ---------------------------------------------------
# Compressed sensing recovery
# ---------------------------------------------------
def compressed_sensing_reconstruct(x, M):
    idx = np.sort(np.random.choice(N, M, replace=False))
    y = x[idx]

    # Measurement operator (rows of IFFT matrix)
    F = np.fft.ifft(np.eye(N)) * N
    A = F[idx,:]

    # Solve: min ||a||_1  s.t. A a = y
    a = cp.Variable(N, complex=True)
    objective = cp.Minimize(cp.norm1(a))
    constraints = [A @ a == y]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    a_hat = a.value
    x_hat = np.real(np.fft.ifft(a_hat)) * N
    return x_hat, idx

# ---------------------------------------------------
# Interactive update
# ---------------------------------------------------
def update(M=40):
    fs_eff = M  # effective sampling frequency

    # --- Uniform sampling ---
    step = N // M
    idx_uniform = np.arange(0, N, step)[:M]
    x_samples = x_true[idx_uniform]
    x_sinc = sinc_reconstruct(t, idx_uniform, x_samples, N)

    # --- Compressed sensing ---
    x_cs, idx_random = compressed_sensing_reconstruct(x_true, M)

    # --- Build Plotly figure ---
    fig = go.Figure()

    # Original
    fig.add_trace(go.Scatter(x=t, y=x_true, mode='lines',
                             name="Original signal", line=dict(color='black')))

    # Sinc reconstruction
    fig.add_trace(go.Scatter(x=t, y=x_sinc, mode='lines',
                             name="Sinc reconstruction (uniform)", 
                             line=dict(color='blue', dash='dash')))

    # CS reconstruction
    fig.add_trace(go.Scatter(x=t, y=x_cs, mode='lines',
                             name=f"CS reconstruction (random)", 
                             line=dict(color='orange', dash='dash')))

    # Uniform samples (blue dots)
    fig.add_trace(go.Scatter(x=idx_uniform, y=x_true[idx_uniform], mode='markers',
                             name="Uniform samples", marker=dict(color='blue', size=8)))

    # Random samples (red dots)
    fig.add_trace(go.Scatter(x=idx_random, y=x_true[idx_random], mode='markers',
                             name="Random samples", marker=dict(color='red', size=8)))

    # Top annotation (Nyquist info)
    fig.add_annotation(
        x=0.5, y=max(x_true)+0.5,
        text=f"M = {M} → fs = {fs_eff} Hz | Nyquist = {2*f_max} Hz | f_max = {f_max} Hz",
        showarrow=False, font=dict(color="green", size=12), xref="paper", yref="y"
    )

    # Bottom annotation (number of sampling points)
    fig.add_annotation(
        x=0.5, y=-0.25, xref="paper", yref="paper",
        text=f"Number of sampling points (uniform & CS) = {M}",
        showarrow=False, font=dict(color="darkblue", size=12)
    )

    fig.update_layout(
        title=f"Uniform vs. CS Reconstruction (M={M} samples)",
        xaxis_title="Time index",
        yaxis_title="Amplitude",
        height=650,
        margin=dict(b=80),  # leave space for bottom annotation
        legend=dict(x=0.01, y=0.99)
    )
    fig.show()

# ---------------------------------------------------
# Interactive widget
# ---------------------------------------------------
interact(update, M=IntSlider(min=10, max=N, step=5, value=40))


interactive(children=(IntSlider(value=40, description='M', max=256, min=10, step=5), Output()), _dom_classes=(…

<function __main__.update(M=40)>

In [ ]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

# ---------------------------------------------------
# Define original signal (sum of 5 sinusoids)
# ---------------------------------------------------
frequencies = [3, 7, 12, 18, 25]   # Hz
amplitudes  = [1, 0.8, 0.6, 0.5, 0.4]

N = 2000  # total number of "dense" time points (acts like fs=2000 Hz for ground truth)
t_cont = np.linspace(0, 1, N, endpoint=False)
x_cont = sum(a * np.sin(2*np.pi*f*t_cont) for a, f in zip(amplitudes, frequencies))

f_max = max(frequencies)
nyquist = 2 * f_max

# ---------------------------------------------------
# Sinc reconstruction
# ---------------------------------------------------
def sinc_reconstruct(t, t_samples, x_samples, fs):
    T = 1/fs
    return np.sum(x_samples * np.sinc((t[:, None] - t_samples[None, :]) / T), axis=1)

# ---------------------------------------------------
# Compressed sensing reconstruction
# ---------------------------------------------------
def cs_reconstruct(x, M):
    N = len(x)
    # Pick M random time samples
    idx = np.sort(np.random.choice(N, M, replace=False))
    y = x[idx]

    # Fourier basis (DFT matrix)
    F = np.fft.fft(np.eye(N)) / np.sqrt(N)  # unitary DFT
    A = F[idx, :]  # measurement operator

    # Solve: min ||a||_1 s.t. A a = y
    a = cp.Variable(N, complex=True)
    objective = cp.Minimize(cp.norm1(a))
    constraints = [A @ a == y]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=False)

    a_hat = a.value
    x_hat = np.real(np.fft.ifft(a_hat) * np.sqrt(N))

    # --- Fix global sign ambiguity ---
    sign = np.sign(np.dot(x_hat, x))
    if sign != 0:
        x_hat = sign * x_hat

    return x_hat, idx

# ---------------------------------------------------
# Interactive update function
# ---------------------------------------------------
def update(fs=80, M=60):
    # --- Uniform sampling (Shannon) ---
    t_samples = np.arange(0, 1, 1/fs)
    x_samples = sum(a * np.sin(2*np.pi*f*t_samples) for a, f in zip(amplitudes, frequencies))
    x_rec_sinc = sinc_reconstruct(t_cont, t_samples, x_samples, fs)

    # --- Compressed sensing (random) ---
    x_rec_cs, idx_random = cs_reconstruct(x_cont, M)

    # --- Plot ---
    fig = go.Figure()

    # Original continuous signal
    fig.add_trace(go.Scatter(x=t_cont, y=x_cont, mode='lines',
                             name="Original signal", line=dict(color='black')))

    # Shannon sinc reconstruction
    fig.add_trace(go.Scatter(x=t_cont, y=x_rec_sinc, mode='lines',
                             name=f"Sinc reconstruction (fs={fs} Hz)", 
                             line=dict(color='blue', dash='dash')))

    # CS reconstruction
    fig.add_trace(go.Scatter(x=t_cont, y=x_rec_cs, mode='lines',
                             name=f"CS reconstruction (M={M} random samples)",
                             line=dict(color='orange', dash='dot')))

    # Uniform sample points
    fig.add_trace(go.Scatter(x=t_samples, y=x_samples, mode='markers',
                             name=f"Uniform samples ({len(t_samples)})", marker=dict(color='blue', size=6)))

    # Random sample points (CS)
    fig.add_trace(go.Scatter(x=t_cont[idx_random], y=x_cont[idx_random], mode='markers',
                             name=f"Random samples (CS: {len(idx_random)})", marker=dict(color='red', size=6)))

    # Top annotation (Nyquist info)
    fig.add_annotation(
        x=0.5, y=max(x_cont)+0.5, xref="paper", yref="y",
        text=f"Nyquist = {nyquist} Hz | f_max = {f_max} Hz",
        showarrow=False, font=dict(color="green", size=12)
    )

    # Bottom annotation (number of samples in both methods)
    fig.add_annotation(
        x=0.5, y=-0.25, xref="paper", yref="paper",
        text=f"Uniform sinc samples = {len(t_samples)} | CS random samples = {len(idx_random)}",
        showarrow=False, font=dict(color="darkblue", size=12)
    )

    fig.update_layout(
        title=f"Shannon vs. Compressed Sensing Reconstruction (fs={fs} Hz, M={M})",
        xaxis_title="Time [s]",
        yaxis_title="Amplitude",
        height=650,
        margin=dict(b=80),  # leave space for bottom annotation
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

# ---------------------------------------------------
# Interactive widget
# ---------------------------------------------------
interact(update,
         fs=widgets.IntSlider(min=10, max=200, step=5, value=80),
         M=widgets.IntSlider(min=20, max=200, step=10, value=60));
